In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from astropy.io import fits
from astropy.table import Table
from astropy.coordinates import SkyCoord
from astropy import units as u
from DS_healpix_fragmentation import radec2pix
from DS_Planck_Unet import val_pix

In [2]:
det_cats = {
    'pz14' : pd.read_csv('/home/rt2122/Data/detected_cats/full_pz14_thr0.1_step8.csv'),
    'pz20' : pd.read_csv('/home/rt2122/Data/detected_cats/full_pz20_thr0.1_step8.csv'),
    'pz25' : pd.read_csv('/home/rt2122/Data/detected_cats/full_pz25_thr0.1_step8.csv'),
    'pz40' : pd.read_csv('/home/rt2122/Data/detected_cats/full_pz40_thr0.1_step8.csv'),
    'pz_act10' : pd.read_csv(
    '/home/rt2122/Data/detected_cats/full_pz_act10_thr0.1_step8.csv'),
    'pz_act14' : pd.read_csv(
    '/home/rt2122/Data/detected_cats/full_pz_act14_thr0.1_step8.csv'),
    'pz_act20' : pd.read_csv('/home/rt2122/Data/detected_cats/full_pz_act20_thr0.1_step8.csv'),
    'pz_act25' : pd.read_csv('/home/rt2122/Data/detected_cats/full_pz_act25_thr0.1_step8.csv')
}

In [3]:
for name in det_cats:
    df = det_cats[name]
    df['pix2'] = radec2pix(df['RA'], df['DEC'], 2)
    df = df[np.in1d(df['pix2'], val_pix)]
    df.index = np.arange(len(df))
    det_cats[name] = df

In [5]:
psz2 = None
with fits.open('/home/rt2122/Data/original_catalogs/psz2.fits') as hdul:
    data = hdul[1].data
    psz2 = Table(data).to_pandas()
mcxc = None
with fits.open('/home/rt2122/Data/original_catalogs/mcxc.fits') as hdul:
    data = hdul[1].data
    mcxc = Table(data).to_pandas()
rm = None
with fits.open('/home/rt2122/Data/original_catalogs/redmapper.fits.gz') as hdul:
    data = Table(hdul[1].data)
    names = [name for name in data.colnames if len(data[name].shape) <= 1]
    rm = data[names].to_pandas()
act = None
with fits.open('/home/rt2122/Data/original_catalogs/act.fits') as hdul:
    data = hdul[1].data
    act = Table(data).to_pandas()

In [6]:
true_cats = {'psz2' : psz2, 'mcxc' : mcxc, 'rm' : rm, 'act' : act}

In [9]:
for name in true_cats:
    df = true_cats[name]
    df['pix2'] = radec2pix(df['RA'], df['DEC'], 2)
    df = df[np.in1d(df['pix2'], val_pix)]
    df.index = np.arange(len(df))
    true_cats[name] = df

<ipython-input-9-5732c1c9f779>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['pix2'] = radec2pix(df['RA'], df['DEC'], 2)


In [8]:
true_cats['mcxc'].rename({'RAdeg' : 'RA', 'DEdeg' : 'DEC'}, axis='columns', inplace=True)
true_cats['act'].rename({'RADeg' : 'RA', 'decDeg' : 'DEC'}, axis='columns', inplace=True)

In [10]:
match_dist = 5 / 60

In [12]:
def calc_error(det_cat, true_cat, shift=15/60, match_dist=5/60, n_try=20, seed=0):
    import numpy as np
    from astropy.coordinates import SkyCoord
    from astropy import units as u
    
    error = []
    np.random.seed(seed)
    for i in range(n_try):
        det_sc = SkyCoord(ra=np.array(det_cat['RA']) * u.degree, 
                          dec=np.array(det_cat['DEC']) * u.degree, frame='icrs')
        angles = np.random.randint(0, 360, len(det_cat))
        det_sc = det_sc.directional_offset_by(angles*u.degree, shift)

        true_sc = SkyCoord(ra=np.array(true_cat['RA']) * u.degree, 
                           dec=np.array(true_cat['DEC']) * u.degree, frame='icrs')
        _, d2d, _ = det_sc.match_to_catalog_sky(true_sc)
        c_error = np.count_nonzero(d2d.degree < match_dist)
        error.append(c_error)
    error = np.array(error)
    return error.mean(), error.std() / np.sqrt(n_try - 1)

In [22]:
comp_df = []
recall_df = []
for det_name in det_cats:
    line = {}
    line_r = {}
    
    sc = SkyCoord(ra=np.array(det_cats[det_name]['RA'])*u.degree, 
                  dec=np.array(det_cats[det_name]['DEC'])*u.degree, frame='icrs')
    
    for tr_name in true_cats: 
        tr_sc = SkyCoord(ra=np.array(true_cats[tr_name]['RA'])*u.degree, 
                      dec=np.array(true_cats[tr_name]['DEC'])*u.degree, frame='icrs')
        idx, d2d, _ = sc.match_to_catalog_sky(tr_sc)
        matched = d2d.degree <= match_dist
        line[tr_name] = np.count_nonzero(det_cats[det_name].iloc[matched]['status'] != 'fn')
        line[tr_name+'_err'], line[tr_name+'_std'] = calc_error(det_cats[det_name], true_cats[tr_name])
        
        line_r[tr_name] = line[tr_name] / len(true_cats[tr_name])
    line['fp'] = np.count_nonzero(det_cats[det_name]['status'] == 'fp')
    line['all'] = len(det_cats[det_name])
    line_r['fp'] = line['fp']
    comp_df.append(pd.DataFrame(line, index=[det_name]))
    recall_df.append(pd.DataFrame(line_r, index=[det_name]))

line = {}
for tr_name in true_cats:
    line[tr_name] = len(true_cats[tr_name])
    line[tr_name+'_err'] = 0
line['fp'] = 0
line['all'] = 0
comp_df.append(pd.DataFrame(line, index=['all']))
comp_df = pd.concat(comp_df)
recall_df = pd.concat(recall_df)

In [23]:
comp_df

,psz2,psz2_err,psz2_std,mcxc,mcxc_err,mcxc_std,rm,rm_err,rm_std,act,act_err,act_std,fp,all
pz14,147,1.55,0.285390,71,1.20,0.304354,135,19.45,1.094183,127,3.90,0.446625,1055,1847
pz20,150,1.30,0.241704,71,1.10,0.203909,144,20.05,0.779592,143,4.45,0.419743,1116,1908
pz25,149,1.05,0.198348,73,1.35,0.195677,151,20.55,1.164101,146,5.35,0.488149,1187,1979
pz40,146,1.55,0.256238,69,0.90,0.250263,142,21.25,0.972991,129,4.40,0.466792,1180,1972
pz_act10,144,0.90,0.216430,64,1.15,0.208693,104,13.10,0.962180,95,2.80,0.321182,624,1313
pz_act14,143,1.25,0.260314,62,0.90,0.176218,114,16.05,0.712575,95,3.45,0.380270,759,1448
pz_act20,132,0.85,0.208693,58,0.85,0.301531,83,12.80,0.899122,72,3.45,0.328273,557,1349
pz_act25,131,0.90,0.190567,58,0.70,0.193309,92,15.15,1.124547,77,3.85,0.342399,624,1416
all,160,0.00,NaN,166,0.00,NaN,3081,0.00,NaN,602,0.00,NaN,0,0


In [24]:
recall_df

,psz2,mcxc,rm,act,fp
pz14,0.91875,0.427711,0.043817,0.210963,1055
pz20,0.93750,0.427711,0.046738,0.237542,1116
pz25,0.93125,0.439759,0.049010,0.242525,1187
pz40,0.91250,0.415663,0.046089,0.214286,1180
pz_act10,0.90000,0.385542,0.033755,0.157807,624
pz_act14,0.89375,0.373494,0.037001,0.157807,759
pz_act20,0.82500,0.349398,0.026939,0.119601,557
pz_act25,0.81875,0.349398,0.029860,0.127907,624
